# Topography Analysis Pipelines
These pipelines take already processed data and create subject/timepoint/condition-specific maps.

### Data-prep pipeline
* Downsample func data to 32k vertices
* Compute distance matrix
* Create vertex-wise correlation matrix

### Network assignment pipeline
* Apply distance cutoffs to create binary matrix
* imput adjacency matrix and distance matrix to infomap algorithm
* Repeat algorithm across 500 random seeds and average

In [ ]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber, FreeSurferSource # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode, JoinNode        # pypeline engine
from pandas import DataFrame, read_csv

#set output file type for FSL to NIFTI_GZ
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
study_home = '/data/perlman/moochie/user_data/CamachoCat/5YOP'
preproc_dir = study_home + '/proc/preprocessing'
hcpproc_dir = study_home + '/proc/hcp_proc'
output_dir = study_home + '/proc/rest_mapping'
workflow_dir = study_home + '/workflows'
#session_info = read_csv(study_home + '/misc/session_info.csv',index_col=None)
#session_info = session_info.astype({'subject_id':str})
#subject_ids = session_info['subject_id'].unique().tolist()
#subject_ids = list(map(str,subject_ids))
subject_ids = ['5000']

proc_cores = 6 # number of cores of processing for the workflows

In [ ]:
# grab data

# create connectivity matrices
def cifti_conn_mat(cifti):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from subprocess import check_call
    
    conn_mat_cifti = cifti.replace('dtseries','dconn')
    check_call(['wb_command -cifti-correlation', cifti, conn_mat_cifti])
    
    return(conn_mat_cifti)

# create distance matrices
def cifti_dist_mat(cifti):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from subprocess import check_call
    import numpy as np
    import nibabel as nib
    from os.path import abspath
    from os import rm
    
    img = nib.load(cifti)
    d = cifti_img.get_fdata()
    n_vertices = d.shape[0]
    
    d_mat = np.zeros((n_vertices,n_vertices))
    
    for vertex in range(0,n_vertices):
        check_call(['wb_command -surface-geodesic-distance',cifti,str(vertex),'d_temp.shape.gii'])
        temp = nib.load('d_temp.shape.gii')
        d_mat[:,vertex]=temp.agg_data()
    
    np.save('dmat.npy',d_mat)
    dist_mat_file=abspath('dmat.npy')
    return(dist_mat_file)


def combine_hemi_dist_mats(left_hemi_mat,right_hemi_mat,fillzeroes):
    
    return(full_dist_mat)

In [ ]:
gifti='/data/perlman/moochie/user_data/CamachoCat/5YOP/proc/hcp_proc/3000/T1w/fsaverage_LR32k/3000.R.midthickness.32k_fs_LR.surf.gii'

from subprocess import check_call
import numpy as np
import nibabel as nib

img = nib.load(gifti)
d = img.agg_data()
n_vertices = d[0].shape[0]

d_mat = np.zeros((n_vertices,n_vertices))

for vertex in range(0,n_vertices):
    check_call(['wb_command','-surface-geodesic-distance',gifti,str(vertex),'d_temp.shape.gii'])
    temp = nib.load('d_temp.shape.gii')
    d_mat[:,vertex]=temp.agg_data()

In [ ]:
d_mat[:10,:10]

In [ ]:
np.savetxt('Rhemi.txt',d_mat)

In [ ]:
import nibabel as nib
import hcp_utils as hcp
import numpy as np

cifti1 = '/data/perlman/moochie/user_data/CamachoCat/5YOP/proc/hcp_proc/3000/MNINonLinear/fsaverage_LR32k/3000.thickness.32k_fs_LR.dscalar.nii'
cifti2 = '/data/perlman/moochie/user_data/CamachoCat/5YOP/5yop/distance1R.shape.gii'
cifti3 = ''
cifti_img = nib.load(cifti2)
#data = cifti_img.get_fdata()
d = cifti_img.agg_data()
d.shape

In [ ]:
make_conn_mat = Node(Function(input_names=['cifti'],output_names=['conn_mat_cifti'],
                              function=cifti_conn_mat),name='make_conn_mat')

make_Ldist_mat = Node(Function(input_names=['cifti'],output_names=['dist_mat_file'],
                               name='make_dist_mat'),name='make_Ldist_mat')

make_Rdist_mat = Node(Function(input_names=['cifti'],output_names=['dist_mat_file'],
                               name='make_dist_mat'),name='make_Rdist_mat')

combine_dist_mats = Node(Function(input_names=['left_hemi_mat','right_hemi_mat','fillzeroes'], 
                                  output_names=['full_dist_mat'],
                                  function=combine_hemi_dist_mats),name='combine_dist_mats')

In [ ]:
# load data
import hcp_utils as hcp
import nibabel as nib
import numpy as np

func_data_file = input_dir +'/{0}/MNINonLinear/Results/movie/movie_Atlas.dconn.nii'.format(subject_id)
func_img = nib.load(func_data_file)
hemi2_func_data = func_img.get_fdata()
hemi1_func_data = hemi2_func_data[:29696,:29696]
hemi2_func_data = hemi2_func_data[29696:59412,29696:59412]

In [ ]:
labels = [str(a) for a in range(0,hemi1_func_data.shape[0])]

nodes_weights = open('movie_h1_nodes_nothresh_weights.txt','a')
y=0
while y < hemi1_func_data.shape[0]:
    for x in range(y,hemi1_func_data.shape[0]-1):
        #z = 0.5*(np.log10(1+hemi1_func_data[x,x+1]) - np.log10(1-hemi1_func_data[x,x+1]))
        #if z>0.05:
        nodes_weights.write('{0} {1} {2}\n'.format(labels[x],labels[x+1],hemi1_func_data[x,x+1]))
    y = y+1
    
    
nodes_weights = open('movie_h2_nodes_nothresh_weights.txt','a')
labels = [str(a + 29696) for a in range(0,hemi2_func_data.shape[0])]   
y=0
while y < hemi2_func_data.shape[0]:
    for x in range(y,hemi2_func_data.shape[0]-1):
        #z = 0.5*(np.log10(1+hemi2_func_data[x,x+1]) - np.log10(1-hemi2_func_data[x,x+1]))
        #if z>0.05:
        nodes_weights.write('{0} {1} {2}\n'.format(labels[x],labels[x+1],hemi2_func_data[x,x+1]))
    y = y+1

In [ ]:
import nibabel as nib
import numpy as np
orig_img = nib.load('/data/perlman/moochie/user_data/CamachoCat/5YOP/hcp_proc/3000/MNINonLinear/Results/firstvol/firstvol_Atlas.dtseries.nii')
orig_data = orig_img.get_fdata()
print(orig_data.shape)
new_data = np.zeros(orig_data.shape)

In [ ]:
from igraph import Graph
import random
random.seed(540000)
gnodes = Graph.Read_Ncol('fixation_h1_nodes_nothresh_weights.txt', names=('node1','node2','weight'), weights=True)
mods1 = gnodes.community_infomap(edge_weights='weight',trials=8) 

In [ ]:
import numpy as np
net1_labels = np.array(mods1.membership) + 1
n,c=np.unique(net1_labels,return_counts=True)
print(n)
print(c)

In [ ]:
for i in range(0,len(n)):
    if c[i]<100:
        net1_labels[net1_labels==n[i]]=0

In [ ]:
print(net1_labels.shape)
new_data[0,0:29696] = net1_labels

In [ ]:
new_img = nib.cifti2.cifti2.Cifti2Image(new_data, header = orig_img.header, nifti_header=orig_img.nifti_header)
nib.save(new_img, 'fixation_left_hemi.dtseries.nii')

In [ ]:
gnodes2 = Graph.Read_Ncol('fixation_h2_nodes_nothresh_weights.txt', names=('node1','node2','weight'), weights=True, directed=True)
mods2 = gnodes2.community_infomap(edge_weights='weight',trials=8) 

In [ ]:
net2_labels = np.array(mods2.membership) + 1
n,c=np.unique(net2_labels,return_counts=True)
print(n)
print(c)

In [ ]:
for i in range(0,len(n)):
    if c[i]<100:
        net2_labels[net2_labels==n[i]]=0

In [ ]:
print(net2_labels.shape)
new_data[0,29696:59412] = net2_labels

In [ ]:
new_img = nib.cifti2.cifti2.Cifti2Image(new_data, header = orig_img.header, nifti_header=orig_img.nifti_header)
nib.save(new_img, 'fixation_right_hemi.dtseries.nii')